In [ ]:
import yfinance as yf

from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import os

# Replace 'YOUR_API_KEY' with your Alpha Vantage API key
api_key = os.environ['ALPHA_VANTAGE_API_KEY']
ticker_symbol = 'AAPL'

# Fetch historical data for the past 5 years
ts = TimeSeries(key=api_key, output_format='pandas')
df, meta_data = ts.get_daily(symbol=ticker_symbol, outputsize='full')

# Display the historical data
print(df.head())

In [87]:
class Stock:
    def __init__(self, ticker) -> None:
        self.ticker = yf.Ticker(ticker)
        self.info = self.ticker.info
        self.market_cap = self.info['marketCap']
        self.quarterly_income_stmt = self.ticker.quarterly_income_stmt
        self.quarterly_balance_sheet = self.ticker.quarterly_balance_sheet
        self.quarterly_cash_flow = self.ticker.quarterly_cash_flow

### (1) Market Cap at or below Net-Current-Asset-Value

In [122]:
msft = Stock("MSFT")

msft.market_cap
total_assets = msft.quarterly_balance_sheet.loc['Total Assets'][0]
total_liabilities =msft.quarterly_balance_sheet.loc['Current Liabilities'][0]

total_assets-total_liabilities

320993000000.0

### (2) Average 5Y annual Free Cash Flow <= 10%

In [ ]:
ba = Stock("BA")

ba.ticker.get_cash_flow(freq='quarterly')

### (3) Positive TTM Free-cashflow

In [118]:

cn = Stock("CNI")
operating_cash_flow_df = cn.ticker.quarterly_cash_flow.loc['Operating Cash Flow']
capital_expendatures_df = cn.ticker.quarterly_cash_flow.loc['Capital Expenditure']
df['TTM']

KeyError: 0

### (4) Zero or negative net-debt

In [ ]:
rbc = Stock("RY.TO")
rbc.quarterly_balance_sheet.loc['Net Debt']

### (5) Sum 5Y dividends or buybacks present

In [ ]:
jpm = yf.Ticker("JPM")
dividend_df = jpm.history(period="5y")["Dividends"]

dividends = dividend_df[dividend_df > 0].to_list()
dividends_paid = (dividend_df > 0).sum()
dividends

### (6) Only companies from selected country HQs or exchanges

In [ ]:
amzn = Stock("AMZN")
country = amzn.info['country']
country